In [ ]:
# TODO: correct the calculation and drop the DPMM part
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import ticker as mticker
import seaborn as sns
from math import sqrt
from numba import njit, float64
from numba.experimental import jitclass
from ing_theme_matplotlib import mpl_style

mpl_style(True)

In [ ]:
# auxiliary functions

In [ ]:
########################################
#              CPMM class              #
########################################

CPMM_spec = [("X", float64), ("Y", float64), ("fee", float64)]


@jitclass(CPMM_spec)
class CPMM:
    """
    This is standard Uniswap V2-style CPMM.
    Fee is charged on input side.
    """

    def __init__(self, p=1_000.0, l=1_000.0, fee=0.003):
        self.X = l / sqrt(p)
        self.Y = l * sqrt(p)
        self.fee = fee

    def price(self):
        """
        spot price of pool
        """
        return self.Y / self.X

    def liquidity(self):
        """
        liquidity of pool
        """
        return sqrt(self.X * self.Y)

    def trade(self, P_ext, c):
        """
        P_ext: external price
        c: tx cost
        """
        new_X = self.X
        new_Y = self.Y

        # price is high enough
        if P_ext * (1 - self.fee) > self.price():
            P_rep = P_ext * (1 - self.fee)
            new_X = self.liquidity() / sqrt(P_rep)
            new_Y = self.Y + (self.liquidity() * sqrt(P_rep) - self.Y) / (1 - self.fee)

        # price is low enough
        elif P_ext < self.price() * (1 - self.fee):
            P_rep = P_ext / (1 - self.fee)
            new_Y = self.liquidity() * sqrt(P_rep)
            new_X = self.X + (self.liquidity() / sqrt(P_rep) - self.X) / (1 - self.fee)

        # price is within non-trading range
        else:
            return 0

        # calculate the change in reserves
        delta_X = new_X - self.X
        delta_Y = new_Y - self.Y

        # calculate the arbitrager's profit
        profit = -(P_ext * delta_X + delta_Y) - c

        if profit > 0:
            # update the state
            self.X = new_X
            self.Y = new_Y

            # return the profit
            return profit
        else:
            return 0

In [ ]:
########################################
#              FMAMM class             #
########################################

FMAMM_spec = [("X", float64), ("Y", float64), ("fee", float64)]


@jitclass(FMAMM_spec)
class FMAMM:
    """
    This is FM-AMM based on constant product invariant.
    Fee is charged on both side to keep the price constant even after adding fee.
    """

    def __init__(self, p=1_000.0, l=1_000.0, fee=0.0015):
        self.X = l / sqrt(p)
        self.Y = l * sqrt(p)
        self.fee = fee

    def price(self):
        return self.Y / self.X

    def liquidity(self):
        return sqrt(self.X * self.Y)

    def trade(self, P_ext, c):
        """
        TODO
        P_ext: external price
        c: tx cost
        """
        pass